![logo](../../_static/images/NCI_logo.png)

-------

# Calculate CMIP6 monthly anomaly and Nino34Index using CDO


In this notebook:

- Load CDO module
- Look at the information of the file
- calculate monthly anomaly
- calculate Nino34Index
- visulise the data using ncview 

The following material uses Coupled Model Intercomparison Project (CMIP6) collections. The CMIP6 terms of use are found [here](https://pcmdi.llnl.gov/CMIP6/TermsOfUse/TermsOfUse6-1.html). For more information on the collection, please [click here](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f6600_2266_8675_3563).


CDO stands for "Cliamte Data Operators". CDO is a collection of command line Operators to manipulate and analyse Climate and NWP model Data.
Supported data formats are GRIB 1/2, netCDF 3/4, SERVICE, EXTRA and IEG. There are more than 600 operators available. See [here](https://code.mpimet.mpg.de/projects/cdo) for more information about this library.

### Load CDO module from Raijin

```
$ module load cdo
```

### List the path of the CMIP6 data

Here we will be using CMIP6 data as an example. Let's see if this file exists, or whether the file path is correct.

```
$ ls /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc
```
This file includes the near surface temperature from the 20th century all-forcing historical simulation based on CESM2.0 model of NCAR. 

### Have a look at the data file using cdo info

Basic usage:
```
$ cdo info <filename> | less
```

**less** display only one page at a time in the termial. You can move forwards and backwards to see more. Press **q** to quit the view.

![1](images/cdo_anomoly_nino1.png)

Use **ncview** to show the data:
```
$ ncview /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc
```

The data looks like this 

![3](images/cdo_anomoly_nino3.png)

### Let's see which year this file includes

We use function **showyear** to display all the years in this file

Basic usage:
```
$ cdo showyear <file>
```

![2](images/cdo_anomoly_nino2.png)

### Calculate monthly anomaly 

In this exercise, we are going to calculate the monthly anomoly between 1971 and 2000. A few steps are in the following order:

1. choose climatology years 1971-2000 
2. calculate the monthly average during 1971-2000
3. Subtract the monthly average from the original file to obtain the difference, i.e., anomaly.

We need three functions to implement this procedure as below

```
$ cdo selyear, <year-range> <input.nc> <output.nc>
```
Select the years and generate a new file.

```
$ cdo ymonmean <input.nc> <output.nc>
```
Get the mean value of each month

```
$ cdo ymonsub <input1.nc> <input2.nc> <output.nc> 
```

Get the difference between two input files and save an output file.

However, if we do three steps, we will generate unecessary intermediate files. CDO allows piping multiple commands in a row. A way to chain these commands together is as the following:

```
$ cdo ymonsub input.nc -ymonmean -selyear,1971/2000 input.nc output.nc
```

**-ymonmean -selyear,1971/2000 input.nc** is treated as the monthly average file. 

```
$ cdo ymonsub /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc -ymonmean -selyear,1971/2000 /g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc tas_Amon_CESM2_historical_r1i1p1f1_gn_197101-200012_anomoly.nc
```

### Let's have a look at the monthly anomoly

```
$ module load ncview
$ ncview tas_Amon_CESM2_historical_r1i1p1f1_gn_197101-200012_anomaly.nc
```

![4](images/cdo_anomoly_nino4.png)

### Calculate Nino34 Index using monthly anomaly

To calcuate Nino34 Index, we need to choose the Nino34 region with lat/lon bbox = (-170,-120,-5,5)

```
$ cdo sellonlatbox,-170,-120,-5,5 tas_Amon_CESM2_historical_r1i1p1f1_gn_197101-200012_anomaly.nc Nino34Region.nc
```
Then use function **fldavg** to calculate the field average, and save the result as the Nino34 index.

```
$ cdo fldavg Nino34Region.nc Nino34Idx.nc
```